# Kaggle Solution
# Dogs vs. Cats Redux: Kernels Edition

In this notebook we will be going to perform classification of Dogs and Cats images. </br>
We have been provided with 25,000 images of training data and 12,500 images of test data. For each image in the test set, you should predict a probability that the image is a dog (1 = dog, 0 = cat)

We first import the necessary libraries

In [1]:
# import the necessary libraries
import cv2
import numpy as np
import pandas as pd
import os
from random import shuffle
import keras
from keras import models
from keras import layers
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
TRAIN_DIR = 'E:/Kaggle/dogs-vs-cats/train/train'         # path to the folder containig train images
TEST_DIR = 'E:/Kaggle/dogs-vs-cats/test/test'            # path to the folder containig test images

IMG_SIZE = 150                                           # image pixel size

We now need to get the labels of the images for training as the labels are not directly provided, we get the labels using the 
<i>image_name</i>. 

In [3]:
# This function returns 0 if the image is cat and returns 1 if dog.
def image_label(img):
    label = img.split('.')[-3]
    if label == 'cat':
        return 0
    elif label == 'dog':
        return 1

The images have to be represented in numbers. This is performed by using the <b>OpenCV</b> library to read and then resizing the image to <b>IMG_SIZE</b>. <br>
This is done for both the traning and test images.

In [4]:
def create_training_data():
    training_data = []
    training_labels = []
    img_paths = os.listdir(TRAIN_DIR)
    shuffle(img_paths)
    for img in img_paths:
        label = image_label(img)
        path = os.path.join(TRAIN_DIR, img)
        img = cv2.resize(cv2.imread(path), (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_CUBIC)
        training_data.append(img)
        training_labels.append(label)
    
    return training_data, training_labels   

In [5]:
def create_testing_data():
    testing_data = []
    testing_num = []
    for img in os.listdir(TEST_DIR):
        img_num = img.split('.')[0]
        path = os.path.join(TEST_DIR, img)
        img = cv2.resize(cv2.imread(path), (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_CUBIC)
        testing_data.append(img)
        testing_num.append(img_num)

    return testing_data, testing_num    

In [6]:
train_data, train_labels = create_training_data() 
test_data, test_num = create_testing_data()

The training data is divided into training and validation data for validating the performance of the model and finally use the 
best model on the test set. <br>

20,000 images are used for training and 5,000 images are used for validation.

In [7]:
train_x = np.array(train_data[:-5000])
train_y = np.array(train_labels[:-5000])

val_x = np.array(train_data[-5000:])
val_y = np.array(train_labels[-5000:])

test_x = np.array(test_data) / 255
test_y = np.array(test_num)

We will be using the Sequential model from the keras library to build the Convolutional Neural Network(CNN). 

In [8]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(32, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flat

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

Prepare generators for training and validation sets

In [10]:
batch_size = 16

train_generator = train_datagen.flow(train_x, train_y, batch_size=batch_size)
validation_generator = val_datagen.flow(val_x, val_y, batch_size=batch_size)

Its now time to start training the model!!! <br>

I have initially ran the model for 15 epochs and found that the model was overfitting after 8 epochs, so I have used 8 epochs.

In [11]:
model.fit_generator(
    train_generator, 
    steps_per_epoch=len(train_x) // batch_size,
    epochs=8,
    validation_data=validation_generator,
    validation_steps=len(val_x) // batch_size
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/8
1250/1250 [==============================] - 614s 491ms/step - loss: 0.6275 - acc: 0.6478 - val_loss: 0.6075 - val_acc: 0.7037
Epoch 2/8
1250/1250 [==============================] - 568s 454ms/step - loss: 0.5270 - acc: 0.7421 - val_loss: 0.4635 - val_acc: 0.7817
Epoch 3/8
1250/1250 [==============================] - 562s 450ms/step - loss: 0.4842 - acc: 0.7723 - val_loss: 0.4524 - val_acc: 0.7869
Epoch 4/8
1250/1250 [==============================] - 562s 450ms/step - loss: 0.4549 - acc: 0.7922 - val_loss: 0.4841 - val_acc: 0.7681
Epoch 5/8
1250/1250 [==============================] - 563s 451ms/step - loss: 0.4335 - acc: 0.8101 - val_loss: 0.4036 - val_acc: 0.8218
Epoch 6/8
1250/1250 [==============================] - 570s 456ms/step - loss: 0.4186 - acc: 0.8147 - val_loss: 0.3920 - val_acc: 0.8232
Epoch 7/8
1250/1250 [==============================] - 565s 452ms/step - loss: 0.3974 - acc: 0.8269 - val_loss: 0.4018 - val_acc: 

We can see that the accuracy on the training set is 83% and accuracy on the test set is also close to 83%, which is a good 
accuracy for such a small trainig set. <br>

Now we finally use the trained model to predict on the test set.

In [18]:
predictions = model.predict(test_x, verbose=1)

12500/12500 [==============================] - 94s 8ms/step


In [22]:
ids = range(1, len(test_x) + 1)
result = pd.DataFrame({"id": ids, "label":list(predictions)})
cols = ['label']

for col in cols:
    result[col] = result[col].map(lambda x: str(x).lstrip('[').rstrip(']')).astype(float)

Write the results into a csv for submission to Kaggle!!!!

In [23]:
result.to_csv("dogs_vs_cats.csv", index = False)